# Palabras Relacionadas - Dataset

En este notebook se explica en detalle el dataset y el procesamiento que requirió para dejarlo listo para la siguiente etapa del proyecto.

En primer lugar, se buscó y seleccionó múltiples textos en formato PDF pertenencientes a distintas materias de la carrera Ingeniería en Sistemas de Información, Universidad Tecnológica Nacional Facultad Regional Mendoza.

Estos, fueron cargados en la carpeta /data/raw.

Se siguió como convención para los nombres el número del año de la materia, seguido de un guión, una abreviatura del nombre de la materia, otro guión y el nombre original del material.

Utilizando algunas librerías de python, se convirtió cada archivo PDF en un archivo txt con su contenido, en /data/plain

In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 4.6 MB/s eta 0:00:00


In [9]:
from pypdf import PdfReader
import os
from os import listdir
from os.path import isfile, join

raw_path = "./data/raw"
plain_path = "./data/plain"

# Arreglo con los nombres de los archivos PDF
raw_files = [os.path.splitext(f)[0] for f in listdir(raw_path) if isfile(join(raw_path, f))]

metrics = {}
for f in raw_files:
    print("\033[94mConvirtiendo archivo: " + f + "\033[0m")

    sf = f.split("-")
    anio = sf[0].strip()
    materia = sf[1].strip()
    if (not anio in metrics):
        metrics[anio] = {}

    if (not materia in metrics[anio]):
        metrics[anio][materia] = 0

    metrics[anio][materia] += os.path.getsize(raw_path + "/" + f + ".pdf")

    reader = PdfReader(raw_path + "/" + f + ".pdf")
    for p in reader.pages:
        with open(plain_path + "/" + f + ".txt", "wb") as t:
            t.write(p.extract_text(
                extraction_mode="plain",
                layout_mode_space_vertically=False).encode("utf-8"))

print("\033[92m")
for a, materias in metrics.items():
    print("Año " + a + ":")
    for m, tamano in materias.items():
        print("\tMateria: " + m + " - " + str(round(tamano/1000000,2)) +"MB")
print("\033[0m")

Convirtiendo archivo: 5 - SSI - Magerit_v3_libro3_guía de técnicas
Convirtiendo archivo: 2 - SO - Tanenbaum
Convirtiendo archivo: 1 - SyO - La organizacion como sistema abierto- Cap 5
Convirtiendo archivo: 1 - SyO - Teoria de sistemas
Convirtiendo archivo: 4 - IO - U6_2_2-INVENTARIO - Articulo ANALISIS ABC
Convirtiendo archivo: 1 - SyO - Resumen SyO U5
Convirtiendo archivo: 4 - ICS - 2.-principiosingenieriasoftware
Convirtiendo archivo: 4 - AS - SNIFFERS Y ESCANEO DE PUERTOS
Convirtiendo archivo: 3 - AP - AdProy_2_Trabajo en Equipo_2022
Convirtiendo archivo: 4 - ICS - SoftwareDesign_PrincipiosyPatrones-Autentia
Convirtiendo archivo: 2 - SO - Stallings
Convirtiendo archivo: 4 - AS - Backups_raids
Convirtiendo archivo: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Convirtiendo archivo: 4 - AS - chiavenato-cap9
Convirtiendo archivo: 3 - AP - respuestas
Convirtiendo archivo: 4 - IO - Breve Resumen ANALISIS SENSIBILIDAD
Convirtiendo archivo: 1 - AyED - Unidades 1 y 2

In [11]:
!zip -r /content/data/plain.zip /content/data/plain

  adding: content/data/plain/ (stored 0%)
  adding: content/data/plain/2 - AS - scrum_manager.txt (deflated 41%)
  adding: content/data/plain/1 - SyO - BPMN 2.0 Manual de referencia.txt (deflated 38%)
  adding: content/data/plain/1 - SyO - Bertoglio - Teoria de sistemas.txt (deflated 54%)
  adding: content/data/plain/2 - SO - ResumenSO.txt (deflated 24%)
  adding: content/data/plain/4 - AS - Elaboracion_de_programas_de_capacitacion.txt (stored 0%)
  adding: content/data/plain/3 - CD - capitulo2.txt (deflated 46%)
  adding: content/data/plain/3 - DS - Eje 1. Metodología y conceptos teóricos aplicados.txt (deflated 52%)
  adding: content/data/plain/1 - SyO - Kendall y Kendall pag. 1 - 13.txt (deflated 54%)
  adding: content/data/plain/1 - AyED - Unidad3 (7929).txt (deflated 40%)
  adding: content/data/plain/1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2.txt (deflated 47%)
  adding: content/data/plain/4 - AS - chiavenato-cap9.txt (defla

A continuación, se tokenizó los archivos planos txt, generando un archivo txt nuevo donde cada línea es un token.

Esto se logró gracias a Spacy.

In [10]:
!pip install spacy
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 37.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from spacy.symbols import ORTH
import re

def is_clean_token(token):
    return not (
        token.is_punct or
        token.is_space or
        token.is_stop or
        len(token.text) == 1)

tokens_path = "./data/tokens"

esp = spacy.load("es_core_news_sm")

for f in raw_files:
    print("\033[94mTokenizando archivo: " + f + "\033[0m")

    with open(plain_path + "/" + f + ".txt", "rb") as pf:
        txt = pf.read().decode("utf-8")
        tokens = esp.tokenizer(txt)
        with open(tokens_path + "/" + f + ".txt", "wb") as tf:
            for token in tokens:
                if (is_clean_token(token)):
                    tf.write((token.text + "\n").encode("utf-8"))

Una vez que se tienen los archivos con los tokens, deseamos detectar conceptos dentro de ellos. Estos conceptos formarán luego nuestro vocabulario.

Se utilizará un algoritmo de ventana deslizante para esto.

In [12]:
from itertools import combinations

In [ ]:
concepts_file = "./data/concepts.txt"

window_size = 4

related_table = {}

with open(concepts_file, "wb") as cf:
    for f in raw_files:
        print("\033[94mDetectando conceptos en archivo: " + f + "\033[0m")

        tokens = []

        with open(tokens_path + "/" + f + ".txt", "rb") as tf:
            tokens = tf.read().decode("utf-8").split("\n")

        for i in range(len(tokens) - window_size):
            window = tokens[i:i+window_size]

            def get_subarrays(arr):
                result = []
                n = len(arr)
                for r in range(1, n+1):  # sizes from 1 to n
                    for indices in combinations(range(n), r):
                        subarray = [arr[i] for i in indices]
                        result.append(subarray)
                return result

            arrays = [arr.insert(0, window[0]) for arr in get_subarrays(window[1,])]

            for arr in arrays:
                if (not arr in related_table):
                    related_table[arr] = 0
                related_table[arr] += 1

print("\033[92mCantidad de conceptos candidatos:" + len(related_table) + "\033[0m")

En el diccionario related_table está contenido cuántas veces aparecieron ciertos tokens en la ventana en cierto orden.

A continuación, en vocabulary, se almacenarán solo los que superen una frecuencia mínima y no otra máxima.

In [ ]:
min_freq = 5
max_freq = 20

vocabulary = {}

for tokens, freq in related_table.items():
    if (freq >= min_freq and freq <= max_freq):
        vocabulary.append(tokens)

print("\033[92mTamaño del vocabulario:" + len(vocabulary) + "\033[0m")

En este punto, ya hemos seleccionado parejas de tokens que suelen aparecer cerca entre sí y en un orden específico.

Estas parejas serán los conceptos, y pasarán a formar nuestro vocabulario.

Ahora, se debe tokenizar nuevamente los textos planos, utilizando los conceptos.

Para esto, se recorrerá cada archivo de /data/plain mediante una ventana deslizante del mismo tamaño utilizado para detectar conceptos, separando en palabras siempre y cuando no se encuentre dentro de la ventana las palabras de un concepto.

Se generarán nuevos tokens, siendo estos numéricos (/data/tokens_num). Los -1 implican que no se detectó un concepto, mientras que los positivos (o 0) corresponden al índice de un concepto en el vocabulario.

A partir de los tokens numéricos, se iterará por cada secuencia de token con un nuevo tamaño de ventana, mayor, tratando de distinguir conceptos relacionados.

Esta ventana se centrará en cada token (no -1), almacenando en un diccionario el token central, los tokens en el contexto y ejemplos negativos (para evitar que la red neuronal, al entrenar, aprenda que todos los tokens siempre están relacionados).

Hecho esto, podemos finalmente armar nuestro dataset. El mismo retornará (mediante get_item()) un centro, su contexto y sus ejemplos negativos.

A partir de este Dataset, a su vez, se generará un DataLoader.

In [ ]:
import torch

A continuación, se armará la estructura de la red neuronal mediante skipgram, utilizando capas Embedding de pytorch.

Como función de pérdida, se utilizará entropía cruzada binaria (Sigmoidea). Esto es así pues requerimos clasificar dos conceptos según si están o no relacionado.

Se optó por abarcar todo el entrenamiento en una misma función. La misma incluye la inicialización de variables y el ciclo de entrenamiento en sí.

Se generó una función auxiliar para el entrenamiento por medio de GPU, en caso de estar disponible.

In [ ]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

Finalmente, se realizó el entrenamiento:

Para verificar la funcionalidad final que buscamos en el proyecto, se planteó la siguiente función:

Algunos ejemplos de la misma serían: